# Crime Analysis Notebook
### This notebook contains the steps used to set up the .aprx file for the analysis.
### This notebook is divided by the following sections:
- Creating Regions
- Join Tables
- Analysis

In [20]:
import arcpy
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"C:\Users\viver\OneDrive\Desktop\crime_analysis\crime_analysis.gdb"

## Creating Regions

In [6]:
regions = {'Northeast': ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'New Jersey', 'Pennsylvania'],
           'Midwest': ['Ohio', 'Michigan', 'Indiana', 'Illinois', 'Wisconsin', 'Minnesota', 'Iowa', 'Missouri', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas'],
           'South': ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Alabama', 'Mississippi', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas'],
           'West': ['Montana', 'Idaho', 'Wyoming', 'Colorado', 'New Mexico', 'Arizona', 'Utah', 'Nevada', 'Washington', 'Oregon', 'California', 'Alaska', 'Hawaii']
}

In [ ]:
# Test code to find region for a state
target = "Maine"
for key, values in regions.items():
    if target in values:
        print(f"{key}")
        break

Northeast


In [4]:
# Add "Region" field to USA_States feature class
fc = "USA_States"
arcpy.management.AddField(fc, "Region", "TEXT")
fields = arcpy.ListFields(fc)
for field in fields:
    print(f"{field.name} - {field.type}")

OBJECTID - OID
Shape - Geometry
STATE_ABBR - String
STATE_FIPS - String
STATE_NAME - String
POPULATION - Integer
POP_SQMI - Double
SQMI - Double
Shape_Length - Double
Shape_Area - Double
Region - String


In [22]:
# Update "Region" field based on "STATE_NAME" field
fields = ["Region", "STATE_NAME"]
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        state_name = row[fields.index("STATE_NAME")]
        for region, states in regions.items():
            if state_name in states:
                row[fields.index("Region")] = region
                cursor.updateRow(row)
                break

## Join Tables

In [ ]:
# Join political party data to states feature class
POLITICAL_PARTY_TABLE = r"C:\Users\viver\Documents\git\gis-portfolio\projects\crime-analysis\data\final\state_majority_party.csv"
states_political_joined_table = arcpy.management.AddJoin(fc, "STATE_NAME", POLITICAL_PARTY_TABLE, "State")
arcpy.management.CopyFeatures(states_political_joined_table, "USA_States_Political_Join")

<Result 'C:\\Users\\viver\\OneDrive\\Desktop\\crime_analysis\\crime_analysis.gdb\\USA_States_Joined'>

In [ ]:
# Join crime data to previously joined feature class
CRIME_DATA_TABLE = r"C:\Users\viver\Documents\git\gis-portfolio\projects\crime-analysis\data\raw\Violent crime and solved crime by state, 2024 - Sheet1.csv"
in_feature = "USA_States_Political_Join"
states_crime_joined_table = arcpy.management.AddJoin(in_feature, "USA_States_STATE_NAME", CRIME_DATA_TABLE, "State")
arcpy.management.CopyFeatures(states_crime_joined_table, "USA_States_Crime_Join")

## Analysis

In [ ]:
# Add fields for percent change increase/decrease in violent crime
in_table = "USA_States_Crime_Join"
arcpy.management.AddField(in_table, "Crime_Pct_Change_Direction", "TEXT")
arcpy.management.AddField(in_table, "Abs_Percent_Change", "DOUBLE")

<Result 'C:\\Users\\viver\\OneDrive\\Desktop\\crime_analysis\\crime_analysis.gdb\\USA_States_Crime_Join'>

In [ ]:
# Calculate new fields based on existing data
in_table = "USA_States_Crime_Join"
direction_expression = "IF(!Crime_Pct_Change! > 0, 'Increase', IF(!Crime_Pct_Change! < 0, 'Decrease', 'No Change'))"
abs_expression = "abs(!Crime_Pct_Change!)"
arcpy.management.CalculateField(in_table, "Crime_Pct_Change_Direction", direction_expression, "PYTHON3")
arcpy.management.CalculateField(in_table, "Abs_Percent_Change", abs_expression, "PYTHON3")

In [17]:
# Run summary statistics on the joined feature class
in_table = "USA_States_Crime_Join"
out_table = "Crime_Summary_Statistics"
statistics_fields = [["Violent_crime_and_solved_crime_by_state__2024___Sheet1_csv_Viole", "MEAN"],
                      ["Violent_crime_and_solved_crime_by_state__2024___Sheet1_csv_Viole", "MEDIAN"],
                      ["Violent_crime_and_solved_crime_by_state__2024___Sheet1_csv_Viole", "VARIANCE"],
                      ["Violent_crime_and_solved_crime_by_state__2024___Sheet1_csv_Solve", "MIN"],
                      ["Violent_crime_and_solved_crime_by_state__2024___Sheet1_csv_Solve", "MAX"],
]
case_field = "USA_States_Political_Join_state_majority_party_csv_Majority_Part"
arcpy.analysis.Statistics(in_table, out_table, statistics_fields, case_field)

<Result 'C:\\Users\\viver\\OneDrive\\Desktop\\crime_analysis\\crime_analysis.gdb\\Crime_Summary_Statistics'>

In [19]:
field = "0.26%"
new_field = field[:-1]  # Remove the last character (%)
print(new_field)

0.26
